1. create folder in local runtime
2. upload dataset zip to local runtime
3. unzip

In [ ]:
!unzip /content/dataset_cropped_224x224_newest.zip -d /content/dataset_cropped_224x224_newest

Archive:  /content/dataset_cropped_224x224_newest-20251128T144454Z-1-001.zip
  inflating: /content/dataset_cropped_224x224_newest/dataset_cropped_224x224_newest/train/b--50-_jpg.rf.fa7945cbc17bad7cb2a6f51a138c0b99_tile1.jpg  
  inflating: /content/dataset_cropped_224x224_newest/dataset_cropped_224x224_newest/train/125_jpg.rf.f37f515011dff4fbb0b882e824432b93_tile2.jpg  
  inflating: /content/dataset_cropped_224x224_newest/dataset_cropped_224x224_newest/train/a--32-_jpg.rf.f7a6bf6256f4a4bc417a23dd47536c7f_tile1.jpg  
  inflating: /content/dataset_cropped_224x224_newest/dataset_cropped_224x224_newest/train/a--87-_jpg.rf.e99dd21bd33d8f9c6b36c8d8b908f5f8_tile2.jpg  
  inflating: /content/dataset_cropped_224x224_newest/dataset_cropped_224x224_newest/train/c--142-_jpg.rf.fb5f460875d029f6681a6b655358776e_tile2.jpg  
  inflating: /content/dataset_cropped_224x224_newest/dataset_cropped_224x224_newest/train/156_jpg.rf.746421f245b72dbf541074bf5d373bdd_tile0.jpg  
  inflating: /content/dataset_crop

In [4]:
!pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 57.8 MB/s eta 0:00:00


In [5]:
import os
import time
import torch
import cv2
import pandas as pd
import numpy as np

from torch.utils.data import Dataset, DataLoader
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
import torch.nn as nn

import albumentations as A
from torchmetrics.detection.mean_ap import MeanAveragePrecision

In [6]:
thop_available = False
try:
    # Check if thop is installed, if not, install it
    import importlib.util
    if not importlib.util.find_spec("thop"):
        print("Installing thop...")
        !pip install thop
        import thop # Re-import after installation attempt
    from thop import profile
    thop_available = True
except Exception as e:
    print(f"THOP library import failed: {e}")
    thop_available = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

class MicroplasticsDataset(Dataset):
    def __init__(self, df, images_dir, transforms=None):
        self.df = df
        self.images_dir = images_dir
        self.transforms = transforms
        self.image_files = df['filename'].unique().tolist()

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_file = self.image_files[idx]
        img_path = os.path.join(self.images_dir, img_file)

        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        records = self.df[self.df['filename'] == img_file]
        boxes = records[['xmin','ymin','xmax','ymax']].values
        labels = records['label'].values

        if self.transforms:
            aug = self.transforms(image=img, bboxes=boxes, class_labels=labels)
            img = aug['image']
            boxes = aug['bboxes']
            labels = aug['class_labels']

        img_tensor = torch.from_numpy(img).permute(2,0,1).float() / 255.0
        boxes_tensor = torch.tensor(boxes, dtype=torch.float32)
        labels_tensor = torch.tensor(labels, dtype=torch.int64)

        return img_tensor, {"boxes": boxes_tensor, "labels": labels_tensor}


def collate_fn(batch):
    images, targets = zip(*batch)
    return list(images), list(targets)

crop_train_dir = "/content/dataset_cropped_224x224_newest/dataset_cropped_224x224_newest/train"
crop_val_dir   = "/content/dataset_cropped_224x224_newest/dataset_cropped_224x224_newest/valid"

train_df = pd.read_csv(os.path.join(crop_train_dir, "_annotations.csv"))
val_df   = pd.read_csv(os.path.join(crop_val_dir, "_annotations.csv"))

# encodings
unique_classes = sorted(train_df['class'].unique())
class_to_idx = {cls: idx+1 for idx, cls in enumerate(unique_classes)}

train_df["label"] = train_df["class"].map(class_to_idx)
val_df["label"]   = val_df["class"].map(class_to_idx)

train_transforms = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.2),
    A.Rotate(limit=30, p=0.5)
], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels']))

train_dataset = MicroplasticsDataset(train_df, crop_train_dir, transforms=train_transforms)
val_dataset   = MicroplasticsDataset(val_df, crop_val_dir, transforms=None)

# Increased val_loader batch_size from 1 to 16 for faster validation
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True,  collate_fn=collate_fn, num_workers=1)
val_loader   = DataLoader(val_dataset,   batch_size=1,  shuffle=False, collate_fn=collate_fn, num_workers=1)

print(f"Loaded {len(train_dataset)} train and {len(val_dataset)} validation images.")

Installing thop...
Using device: cuda
Loaded 1869 train and 647 validation images.


In [ ]:
import os

train_files_fs = [f for f in os.listdir(crop_train_dir) if f.lower().endswith((".jpg", ".png", ".jpeg"))]
val_files_fs   = [f for f in os.listdir(crop_val_dir)   if f.lower().endswith((".jpg", ".png", ".jpeg"))]

print("train images on disk:", len(train_files_fs))
print("val images on disk:",   len(val_files_fs))
print("total on disk:",        len(train_files_fs) + len(val_files_fs))

print("train in CSV:", train_df["filename"].nunique())
print("val in CSV:",   val_df["filename"].nunique())


train images on disk: 2308
val images on disk: 816
total on disk: 3124
train in CSV: 1869
val in CSV: 647


# Model Setup

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


class EfficientNetBackbone(nn.Module):
    """
    Use EfficientNet-B0 as a feature extractor backbone for Faster R-CNN.
    Output feature map has 1280 channels for EfficientNet-B0.
    """
    def __init__(self, weights=EfficientNet_B0_Weights.IMAGENET1K_V1):
        super().__init__()
        base = efficientnet_b0(weights=weights)
        self.features = base.features
        self.out_channels = 1280  # EfficientNet-B0 final feature channels

    def forward(self, x):
        return self.features(x)


backbone = EfficientNetBackbone()

# RPN anchors (you can tweak these if objects are very small/large)
anchor_generator = AnchorGenerator(
    sizes=((16, 32, 64, 128, 256),),
    aspect_ratios=((0.5, 1.0, 2.0),),
)

# One foreground class ("microplastic") + background
# If you have multiple classes, change this accordingly.
num_classes = 2

model = FasterRCNN(
    backbone,
    num_classes=num_classes,
    rpn_anchor_generator=anchor_generator,
    min_size=224,   # adapt if your crops are different size
    max_size=224,
)

model = model.to(device)

# Optimizer & LR scheduler
optimizer = optim.AdamW(
    model.parameters(),
    lr=1e-4,
    weight_decay=1e-4,
)

lr_scheduler = optim.lr_scheduler.StepLR(
    optimizer,
    step_size=5,
    gamma=0.1,
)


Using device: cuda
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth


100%|██████████| 20.5M/20.5M [00:00<00:00, 239MB/s]


In [ ]:
import os
import time
import pandas as pd
from torchmetrics.detection.mean_ap import MeanAveragePrecision
try:
    from thop import profile
    thop_available = True
except Exception as e:
    print(f"THOP not available, MACs will be skipped: {e}")
    thop_available = False

base_dir = "/models/efficientnet-b0"
os.makedirs(base_dir, exist_ok=True)

num_epochs = 30
metric = MeanAveragePrecision(iou_type="bbox").to(device)

history = []

print(f"Starting training for {num_epochs} epochs...")
train_start_time = time.time()

for epoch in range(1, num_epochs + 1):
    model.train()
    epoch_losses = []
    for images, targets in train_loader:
        images  = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)
        loss = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_losses.append(loss.item())

    avg_loss = sum(epoch_losses) / max(len(epoch_losses), 1)

    model.eval()
    metric.reset()
    with torch.no_grad():
        for images, targets in val_loader:
            images  = [img.to(device) for img in images]
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            preds = model(images)
            metric.update(preds, targets)

    metrics_dict = metric.compute()
    map_50    = float(metrics_dict["map_50"])
    map_50_95 = float(metrics_dict["map"])

    # Scheduler
    lr_scheduler.step()

    # Лог
    history.append({
        "epoch": epoch,
        "loss": avg_loss,
        "mAP_50": map_50,
        "mAP_50_95": map_50_95,
    })

    print(
        f"Epoch {epoch:02d}/{num_epochs:02d} | "
        f"loss={avg_loss:.4f} | "
        f"mAP@50={map_50:.4f} | "
        f"mAP@50:95={map_50_95:.4f}"
    )

total_time_sec = time.time() - train_start_time
print(f"\nTotal training time: {total_time_sec:.1f} s ({total_time_sec/60:.2f} min)")

macs_G = None
if thop_available:
    model.eval()
    dummy = torch.randn(1, 3, 224, 224).to(device)
    macs, _ = profile(model, inputs=(dummy,), verbose=False)
    macs_G = macs / 1e9
    print(f"MACs (G): {macs_G:.3f}")
else:
    print("MACs (G): N/A (THOP not available)")

total_params = sum(p.numel() for p in model.parameters())
params_M = total_params / 1e6
print(f"Params (M): {params_M:.3f}")

final_model_path = os.path.join(base_dir, "efficientnetb0_fasterrcnn_final.pth")
torch.save(model.state_dict(), final_model_path)
print(f"Final model saved to: {final_model_path}")

df_hist = pd.DataFrame(history, columns=["epoch", "loss", "mAP_50", "mAP_50_95"])
csv_path = os.path.join(base_dir, "efficientnetb0_training_history.csv")
df_hist.to_csv(csv_path, index=False)
print(f"Training history saved to: {csv_path}")


Starting training for 30 epochs...
Epoch 01/30 | loss=0.2149 | mAP@50=0.0152 | mAP@50:95=0.0034
Epoch 02/30 | loss=0.1277 | mAP@50=0.0798 | mAP@50:95=0.0227
Epoch 03/30 | loss=0.1246 | mAP@50=0.1215 | mAP@50:95=0.0380
Epoch 04/30 | loss=0.1251 | mAP@50=0.1719 | mAP@50:95=0.0561
Epoch 05/30 | loss=0.1210 | mAP@50=0.2295 | mAP@50:95=0.0766
Epoch 06/30 | loss=0.1215 | mAP@50=0.2587 | mAP@50:95=0.0869
Epoch 07/30 | loss=0.1213 | mAP@50=0.2647 | mAP@50:95=0.0896
Epoch 08/30 | loss=0.1208 | mAP@50=0.2747 | mAP@50:95=0.0928
Epoch 09/30 | loss=0.1214 | mAP@50=0.2842 | mAP@50:95=0.1013
Epoch 10/30 | loss=0.1200 | mAP@50=0.3105 | mAP@50:95=0.1061
Epoch 11/30 | loss=0.1201 | mAP@50=0.3120 | mAP@50:95=0.1081
Epoch 12/30 | loss=0.1204 | mAP@50=0.3070 | mAP@50:95=0.1084
Epoch 13/30 | loss=0.1192 | mAP@50=0.3104 | mAP@50:95=0.1087
Epoch 14/30 | loss=0.1201 | mAP@50=0.3236 | mAP@50:95=0.1083
Epoch 15/30 | loss=0.1199 | mAP@50=0.3182 | mAP@50:95=0.1096
Epoch 16/30 | loss=0.1194 | mAP@50=0.3153 | mAP@50